In [ ]:
## all imports
from IPython.display import HTML
import numpy as np
import urllib.request
import urllib.error
import bs4 #this is beautiful soup
import time
from datetime import timedelta
import operator
import socket
import pickle
import re # regular expressions

from pandas import Series
import pandas as pd
from pandas import DataFrame

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context("talk")
sns.set_style("white")


In [ ]:
# To get a list of Netflix page title URL's to get values for our custom dataset

netflix_originals = urllib.request.urlopen('https://www.netflix.com/in/browse/genre/839338').read()
netflix_originals_bs4 = bs4.BeautifulSoup(netflix_originals)

netflix_1st_dataframe = pd.DataFrame(None,columns= ['Title','Link','ID','Status'])
netflix_1st_dataframe

for i in netflix_originals_bs4.findAll('section',{'nm-collections-row'}):
    temp = len(i.h1.text)-12
    if ((i.h1.text[temp:(len(i.h1.text))]) == 'Explore more'):
        #print(i.h1.text[0:temp])
        for j in i.findAll('li',{'nm-content-horizontal-row-item'}):
            #print(j.a.get('href'),j.a.text,type(j.a.img.get('data-title-id')))
            if (j.a.text in netflix_1st_dataframe.Title.values):
                netflix_1st_dataframe.Status.loc[netflix_1st_dataframe.Title == j.a.text] = (netflix_1st_dataframe.Status.loc[netflix_1st_dataframe.Title == j.a.text] + '//' +i.h1.text[0:temp])
            else:   
                netflix_1st_dataframe = netflix_1st_dataframe.append({'Title': j.a.text, 'Link': j.a.get('href'), 'ID': int(j.a.img.get('data-title-id')), 'Status': i.h1.text[0:temp] }, ignore_index=True)
    else:
        #print(i.h1.text)
        for j in i.findAll('li',{'nm-content-horizontal-row-item'}):
            #print(j.a.get('href'),j.a.text,type(j.a.img.get('data-title-id')))
            if (j.a.text in netflix_1st_dataframe.Title.values):
                netflix_1st_dataframe.Status.loc[netflix_1st_dataframe.Title == j.a.text] = (netflix_1st_dataframe.Status.loc[netflix_1st_dataframe.Title == j.a.text] + '//' +i.h1.text)
                
            else:
                netflix_1st_dataframe = netflix_1st_dataframe.append({'Title': j.a.text, 'Link': j.a.get('href'), 'ID': int(j.a.img.get('data-title-id')), 'Status': i.h1.text }, ignore_index=True)
    

In [ ]:
# Saving the dataset into a csv file, backed up to your local system
netflix_1st_dataframe.to_csv('out.csv',index=False)

In [ ]:
# Code to scrape and clean the dataset

# Temporary Datframe and Original intilizations; i-Contains_Links//j- Used in loop for parsing ist info in website// z -Contains ID values// v- Run for loop range// k,d- Used for parsing 2nd loop  to meaningful values


# Timimg the code * some code at end for same 
start_time = time.time()

temp_dataframe = pd.DataFrame([[None,None,None,None,None,None,None,None,None,None,None,None]],columns= ['Title','ID','TV/Movie','Year','Age-Restriction','Number of Seasons','Duaration of movies','Audio','Subtitle','Genre','Download','Mood'])
netflix_2nd_dataframe = pd.DataFrame(None,columns= ['Title','ID','TV/Movie','Year','Age-Restriction','Number of Seasons','Duaration of movies','Audio','Subtitle','Genre','Download','Mood'])

len_netflix_1st_dataframe = len(netflix_1st_dataframe)
# Taking link from data frame 
for i,z,v in zip(netflix_1st_dataframe.Link.values[0:len_netflix_1st_dataframe] , netflix_1st_dataframe.ID.values[0:len_netflix_1st_dataframe], range(len_netflix_1st_dataframe)):
#for i,z,v in zip(netflix_1st_dataframe.Link.values[0:5],netflix_1st_dataframe.ID.values[0:5],range(5)):

    
    # Appending dataframe to create flexiblity in adding data 
    netflix_2nd_dataframe = netflix_2nd_dataframe.append(temp_dataframe,ignore_index=True)
    
    
    # Creating Beautifulsoup link 
    temp_link = urllib.request.urlopen(i).read()
    temp_link_bs4 = bs4.BeautifulSoup(temp_link)
    
    
    # 1st part of Netflix Page(Around the title) --- Scrapping 
    
    
    # Assingning restrict_age to populate the dataframe using the text later in the code
    try:
        restrict_age = temp_link_bs4.find('span',{'class':'title-info-metadata-item item-maturity'}).text
    except:
        restrict_age = None 

        
    # Adding year in Netflix 2nd Dataframe
    #netflix_2nd_dataframe['Year'].iloc[v]= int(j.div.contents[0].text)    
    try:
        netflix_2nd_dataframe['Year'].iloc[v]= int(temp_link_bs4.find('span',{'class': 'title-info-metadata-item item-year'}).text)
    except:
        netflix_2nd_dataframe['Year'].iloc[v]= None
        

    # Assigning No_of_Seasons so that we can populate the dataframe using the text later in the code    
    try:
        No_of_Seasons = temp_link_bs4.find('span',{'class': 'title-info-metadata-item item-runtime'}).text
    except:
        No_of_Seasons = None

        
    # Adding Title to netflix 2nd Dataframe
    try:
        netflix_2nd_dataframe['Title'].iloc[v] = temp_link_bs4.find('h1',{'class': 'title-title'}).text
    except:
        netflix_2nd_dataframe['Title'].iloc[v] = None
        
    
    #Adding Id to netflix 2nd Dataframe
    netflix_2nd_dataframe['ID'].iloc[v] = z
    
    
    count_watch_offline = 0
    # 2nd Part of netflix title page More Details --- Scrapping
    for k in temp_link_bs4.findAll('div',{'class': 'more-details-container'}):
        for d in range(len(k.contents)):
            temp_content = k.contents[d].contents[0].text
            
            # Exctartion of more details
            #if( (temp_content == 'Watch offline') or (temp_content == 'This show is...') or (temp_content == 'This movie is...') or (temp_content == 'Audio') or (temp_content == 'Subtitles') or (temp_content == 'Genres')):
                #print(temp_content)
                #print(k.contents[d].contents[1].text)
            
            # Adding Audio, Download, Subtitle, Genre, Mood to netflix 2nd dataframe
            
            # Audio
            if (temp_content == 'Audio'):
                
                # List conversion and dropping whitespaces
                list_for_dataframe = k.contents[d].contents[1].text.split(',')
                list_for_dataframe = [list_for_dataframe_string.strip() for list_for_dataframe_string in list_for_dataframe]
                # Dataframe column --- Audio
                netflix_2nd_dataframe['Audio'].iloc[v] = list_for_dataframe
                
            # Download * code completed at bottom with # Not-Download
            elif (temp_content == 'Watch offline'):
                netflix_2nd_dataframe['Download'].iloc[v] = k.contents[d].contents[1].text
                count_watch_offline = count_watch_offline + 1
            
            # Subtitle
            elif(temp_content == 'Subtitles'):
                
                # List conversion and dropping whitespaces
                list_for_dataframe = k.contents[d].contents[1].text.split(',')
                list_for_dataframe = [list_for_dataframe_string.strip() for list_for_dataframe_string in list_for_dataframe]
                # Dataframe column --- Subtitle
                netflix_2nd_dataframe['Subtitle'].iloc[v] = list_for_dataframe
                
            # Genres
            elif(temp_content == 'Genres'):
                
                # List conversion and dropping whitespaces
                list_for_dataframe = k.contents[d].contents[1].text.split(',')
                list_for_dataframe = [list_for_dataframe_string.strip() for list_for_dataframe_string in list_for_dataframe]
                # Dataframe column --- Genre
                netflix_2nd_dataframe['Genre'].iloc[v] = list_for_dataframe
                
                
            # Mood    
            elif( (temp_content == 'This show is...') or (temp_content == 'This movie is...')):
                
                # List conversion and dropping whitespaces
                list_for_dataframe = k.contents[d].contents[1].text.split(',')
                list_for_dataframe = [list_for_dataframe_string.strip() for list_for_dataframe_string in list_for_dataframe]
                # Dataframe column --- Mood
                netflix_2nd_dataframe['Mood'].iloc[v] = list_for_dataframe
                
            
        
    # Adding no.of Seasons, Length of Duartion and TV/Movie in Netflix 2nd Dataframe
    if No_of_Seasons is not None:
        if (  (len(No_of_Seasons) == 9) or (len(No_of_Seasons) == 8)):
            netflix_2nd_dataframe['Number of Seasons'].iloc[v] = int(No_of_Seasons[0:1])
            netflix_2nd_dataframe['TV/Movie'].iloc[v] = 'TV'
        elif (len(No_of_Seasons) == 10):
            netflix_2nd_dataframe['Number of Seasons'].iloc[v] = int(No_of_Seasons[0:2])
            netflix_2nd_dataframe['TV/Movie'].iloc[v] = 'TV'
        else:
            netflix_2nd_dataframe['Duaration of movies'].iloc[v] = No_of_Seasons
            netflix_2nd_dataframe['TV/Movie'].iloc[v] = 'Movie'
    
    
    # Adding Age Resctriction to Netflix 2nd Dataframe
    if (restrict_age != 'All ') and (restrict_age is not None):
        netflix_2nd_dataframe['Age-Restriction'].iloc[v] = int(restrict_age[0:(len(restrict_age)-2)])
    else:
        netflix_2nd_dataframe['Age-Restriction'].iloc[v] = 0

    
   # Not-Download
    if (count_watch_offline == 0):
        netflix_2nd_dataframe['Download'].iloc[v] = 'Cannot Download'  
        
   # Sleep to avoid requesting link every millisecond     
    time.sleep(1)
    #print()
    #print()
    #print()
    #print()
    
# Timing for print
fin_time = time.time()
print("Program needed(with time.sleep) ---- %.2f" %(fin_time - start_time) )
print("Program actually needs(without time.sleep) ----- %.2f" %(fin_time - start_time - len(netflix_2nd_dataframe)))

In [ ]:
# Saving the dataset into a csv file, backed up to your local system
netflix_2nd_dataframe.to_csv('netlix 2nd dataframe.csv',index=False)

In [ ]:
# Code for visualization of your dataset

Year_Age_Groupby =netflix_2nd_dataframe['Age-Restriction'][netflix_2nd_dataframe['Age-Restriction'] != 0].groupby(netflix_2nd_dataframe['Year'][netflix_2nd_dataframe.Year.notnull()])

Visulization2_mean10_agevsyear = Year_Age_Groupby.count()[Year_Age_Groupby.apply(lambda f: (f.count() > 10))]

Visulization1_count_agevsyear =Year_Age_Groupby.apply(lambda f: (f.count()))



fig = plt.figure(figsize=(20,10),dpi=100,frameon=True)

ax1 = fig.add_subplot(1,2,1)#netflix_2nd_dataframe.Year.unique())

ax2 = fig.add_subplot(1,2,2)
unique_values_of_year_netflix_2 = [str(i) for i in np.sort(netflix_2nd_dataframe.Year[netflix_2nd_dataframe.Year.notnull()].unique())]
unique_values_mean10_of_year_netflix_2 = [str(i) for i in np.sort(Visulization2_mean10_agevsyear.index)]
x = unique_values_of_year_netflix_2
y = Visulization1_count_agevsyear

ax1.bar(x, y)
for i in ax1.patches:
    ax1.text(i.get_x()+0.25, ((i.get_height()))+0.2, str(round((i.get_height()), 2)), fontsize = 15, fontweight ='bold',color ='grey')
ax1.set_ylabel('No. of Originals released per year', color='red')
ax1.set_xlabel('Year')


ax2.bar(unique_values_mean10_of_year_netflix_2,Year_Age_Groupby.apply(lambda f:(f.mean()))[Year_Age_Groupby.apply(lambda f: (f.count() > 10))])
for i in ax2.patches:
    ax2.text(i.get_x()+0.25, ((i.get_height()))+0.2, str(round((i.get_height()), 2)), fontsize = 15, fontweight ='bold',color ='grey') 
ax2.set_ylabel('Min. Age Required(approx)', color='red')
ax2.set_xlabel('Year')